In [21]:
%%capture
# run this cell if this package is not installed
!pip install nbformat
import string

In [18]:
%%capture
%run setup.ipynb

In [22]:
def normalize_document(doc):
    wpt = nltk.WordPunctTokenizer()
    stop_words = set(nltk.corpus.stopwords.words('english'))

    # sentences to be removed
    p1 = "Chat Conversation Start"
    p2 = "Chat Conversation End"
    
    # Remove p1 only the first time it appears
    doc = re.sub(rf'^\s*{re.escape(p1)}\s*', '', doc)

    # Remove p2 only the last time it appears
    doc = re.sub(rf'\s*{re.escape(p2)}\s*$', '', doc)
    
    # remove email addresses
    doc = re.sub(r'\b\S*@\S*\.\S*\b', '', doc)
    # lowercase and remove special characters\whitespace
    doc = re.sub(r'[^a-zA-Z\s]', '', str(doc), re.I | re.A)
    doc = doc.lower()
    doc = doc.strip()
            
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    #remove punctuation
    doc = re.sub(r'[%s]' % re.escape(string.punctuation), '', doc) #remove punctuation
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    
    return doc


In [23]:
df_norm = df['Content'].apply(normalize_document)

In [24]:
df_norm.iloc[0]

'u good girls bed haha omg miss lot lovely friend miss smile long time yah u yes crazy crzy friend hehe im friend house u great came home well want ask everything good heard happened cat judi im really sad hope good spent good time u everything great dont worry fine new news u im excited since im going usa monday see friend u fantastic city u new jersey lot work u long u one week ill back meet u good im happy try safe enjoy lot maybe plan travel later friends still remember traveled last time montreal really fun good time hope repeat soon spend crazy time like soon'

In [25]:
nlp = spacy.load("en_core_web_sm")

In [26]:
def lemmatizer(text):        
    sent = []
    doc = nlp(text)
    #print(doc)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

In [27]:
df["question_lemmatize"] = df['Content'].apply(lambda x: lemmatizer(x))

In [28]:
df

,File_Name,Content,question_lemmatize
0,1.txt,Chat Conversation Start U1: Good girls are in...,chat Conversation Start \n\n u1 : good girl be...
1,11.txt,"Chat Conversation Start U1: Hi, How are you?...","\n Chat Conversation start \n\n u1 : hi , how ..."
2,12.txt,Chat Conversation Start U1: Hello How are y...,\n Chat Conversation Start \n\n\n u1 : hello h...
3,13.txt,Chat Conversation Start U1: I want to as...,\n\n\n\n chat Conversation start \n\n u1 : I...
4,14.txt,"Chat Conversation Start U1: Hi, I checke...",\n\n\n chat Conversation Start \n\n \n u1 : Hi...
...,...,...,...
11331,84444.txt,From: psyrobtw@ubvmsd.cc.buffalo.edu (Robert W...,from : psyrobtw@ubvmsd.cc.buffalo.edu ( Robert...
11332,84445.txt,From: sandvik@newton.apple.com (Kent Sandvik) ...,from : sandvik@newton.apple.com ( Kent Sandvik...
11333,84446.txt,From: sandvik@newton.apple.com (Kent Sandvik) ...,from : sandvik@newton.apple.com ( Kent Sandvik...
11334,84507.txt,From: cutter@gloster.via.mind.org (cutter) Sub...,from : cutter@gloster.via.mind.org ( cutter ) ...
